In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from utils import plot_candlestick

In [3]:
market_df = pd.read_json('../.log/market.json')
flattened_df = pd.json_normalize(market_df.to_dict(orient='records'))

In [4]:
flattened_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23553 entries, 0 to 23552
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   timeframe                  23553 non-null  object 
 1   meta.key                   23553 non-null  object 
 2   meta.timestamp             23553 non-null  float64
 3   meta.priority              23553 non-null  int64  
 4   meta.version               23553 non-null  int64  
 5   meta.group                 23553 non-null  object 
 6   meta.name                  23553 non-null  object 
 7   symbol.name                21689 non-null  object 
 8   symbol.taker_fee           21689 non-null  float64
 9   symbol.maker_fee           21689 non-null  float64
 10  symbol.min_position_size   21689 non-null  float64
 11  symbol.min_price_size      21689 non-null  float64
 12  symbol.position_precision  21689 non-null  float64
 13  symbol.price_precision     21689 non-null  flo

In [5]:
flattened_df.describe()

,meta.timestamp,meta.priority,meta.version,symbol.taker_fee,symbol.maker_fee,symbol.min_position_size,symbol.min_price_size,symbol.position_precision,symbol.price_precision,ohlcv.timestamp,ohlcv.open,ohlcv.high,ohlcv.low,ohlcv.close,ohlcv.volume
count,2.355300e+04,23553.0,23553.0,2.168900e+04,2.168900e+04,21689.000000,21689.000000,21689.000000,21689.000000,2.355300e+04,23553.000000,23553.000000,23553.000000,23553.000000,2.355300e+04
mean,1.693509e+09,4.0,1.0,6.000000e-04,1.000000e-04,0.079875,0.010961,1.406566,2.593434,1.692434e+12,94.003475,94.219412,93.783136,93.997425,1.648516e+04
std,5.601020e+02,0.0,0.0,1.084227e-19,2.710568e-20,0.039843,0.019720,0.804901,0.804901,7.731063e+08,166.189589,166.615192,165.761030,166.177212,4.593352e+04
min,1.693509e+09,4.0,1.0,6.000000e-04,1.000000e-04,0.001000,0.001000,1.000000,1.000000,1.690917e+12,4.395000,4.399000,4.311000,4.395000,0.000000e+00
25%,1.693509e+09,4.0,1.0,6.000000e-04,1.000000e-04,0.100000,0.001000,1.000000,3.000000,1.691800e+12,6.097000,6.103000,6.092000,6.097000,6.411000e+02
50%,1.693509e+09,4.0,1.0,6.000000e-04,1.000000e-04,0.100000,0.001000,1.000000,3.000000,1.692517e+12,20.629000,20.653000,20.606000,20.629000,4.364300e+03
75%,1.693509e+09,4.0,1.0,6.000000e-04,1.000000e-04,0.100000,0.001000,1.000000,3.000000,1.693106e+12,24.456000,24.476000,24.433000,24.456000,1.649340e+04
max,1.693512e+09,4.0,1.0,6.000000e-04,1.000000e-04,0.100000,0.050000,3.000000,3.000000,1.693512e+12,579.700000,580.200000,578.900000,578.900000,2.047545e+06


In [6]:
symbol_dataframes = {}

rename_map = {
    'ohlcv.timestamp': 'timestamp',
    'ohlcv.open': 'open',
    'ohlcv.high': 'high',
    'ohlcv.low': 'low',
    'ohlcv.close': 'close',
    'ohlcv.volume': 'volume'
}

for symbol in flattened_df['symbol.name'].unique():
    if symbol is np.nan:
        continue
    
    symbol_df = flattened_df[flattened_df['symbol.name'] == symbol]
    
    ohlcv_df = symbol_df[list(rename_map.keys())].copy()
    ohlcv_df = ohlcv_df.rename(columns=rename_map)
    ohlcv_df['symbol'] = symbol
    ohlcv_df['timestamp'] = pd.to_datetime(ohlcv_df['timestamp'], unit='ms')
    
    symbol_dataframes[symbol] = ohlcv_df

In [7]:
symbol_dataframes.keys()

dict_keys(['UNIUSDT', 'SOLUSDT', 'YFIIUSDT'])

In [8]:
fig = plot_candlestick(symbol_dataframes)
fig.show()